In [2]:
from scipy.io import FortranFile as ff
import numpy as np
#np.set_printoptions(threshold=50)
import math
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.colors import LogNorm

In [3]:
#Read the initial rays, to get the weights (computed based on the luminosity of the stars)
IC_file = '/cral/mauerhofer/RascasFiles/sphinx/05_F1000/02_IC20_BP/00183/OI_SiII/CD/Halo1/IC_columnDensity'
f = ff(IC_file,'r')
nrays = f.read_ints()[0]
f.read_reals()
weights = f.read_reals()
f.close()

In [4]:
print(nrays)
print(weights)

1510150
[5.89591354e+45 5.65252523e+45 2.36823761e+45 ... 1.39650300e+46
 4.74858049e+45 4.63669529e+45]


In [5]:
#Read the output
output_CD_file = '/cral/mauerhofer/RascasFiles/sphinx/05_F1000/02_IC20_BP/00183/OI_SiII/CD/Halo1/output_CD_2'
f = ff(output_CD_file,'r')
nrays = f.read_ints()[0]
ndirections = f.read_ints()[0]
nGas = f.read_ints()[0]
vmin, vmax = f.read_reals()
nBins = f.read_ints()[0]
dv = (vmax-vmin)/nBins
v = np.linspace(vmin - dv/2, vmax + dv/2, nBins+2)

In [6]:
print(ndirections, nGas, vmin, vmax, nBins)

10 3 -750.0 750.0 32


In [7]:
#Define the directions 
import healpy as hp
nside = 3  #nDirections = 12 * nside^2
nvec = hp.nside2npix(nside)
directions = np.zeros((nvec,3))
for i in range (0,nvec) :
    directions[i,:] = hp.pix2vec(nside,i)

In [8]:
#plt.style.use('seaborn')

In [9]:
#OI column density :
for k in range(ndirections):

    CD = f.read_reals()
    #CD begins with npart CD for the first bin of velocity,  then npart CD for the second bin,  etc

    CD[np.where(CD>0)] = np.log10(CD[np.where(CD>0)])

    #Question : 
    #for i in range(nrays):
    #    weights[i] = weights[i]/np.where(CD2[:,i]!=0)[0].size
    
    v_all = np.repeat(v,nrays)
    weights_all = np.tile(weights,nBins+2)

    xedges = np.linspace(vmin-dv, vmax + dv,nBins+3)
    yedges = np.arange(1,21.1,0.8)

    histo, xedges, yedges = np.histogram2d(v_all, CD, bins=(xedges,yedges), weights=weights_all, normed=False)

    for i in range(histo[:,0].size):
        histo[i,:] = histo[i,:]/sum(histo[i,:])

    histo = np.ma.masked_where(histo < 0.01, histo)
    cmap = plt.cm.gist_ncar_r
    #cmap = plt.cm.PuBu_r
    cmap.set_bad(color='white')
    #cmap.set_bad(color='black')

    fig, ax = plt.subplots(figsize=(10, 6))
    plt.imshow(histo.T, origin='lower', extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]], aspect='auto', cmap=cmap)#,norm=LogNorm())
    plt.axhline(14.5,color='purple',linestyle='--', label=r'$\tau_{1302} = 10$')
    plt.axhline(13.5,color='deeppink',linestyle='--', label=r'$\tau_{1302} = 1$')
    plt.axvline(0,color='limegreen',linestyle='--')
    plt.legend(loc='upper left', fontsize=14)
    cbar = plt.colorbar()
    plt.clim(0,1)
    plt.xlabel('v [km/s]', fontsize=14)
    plt.ylabel(r'$\mathrm{log}( \: N_{OI} \, [\mathrm{cm}^{-2}] \: )$', fontsize=14)
    #plt.title('2D histogram, direction %s'%directions[k], fontsize = 16)
    plt.title('2D histogram, direction %05.4f %05.4f %05.4f'%(directions[k,0],directions[k,1],directions[k,2]), fontsize = 16)
    cbar.set_label('Fraction of luminosity', fontsize=14)
    plt.xticks(np.arange(xedges[0]+dv, xedges[-1]-dv+0.01, 150))
    #plt.show()
    plt.savefig('/cral/mauerhofer/RascasFiles/sphinx/05_F1000/02_IC20_BP/00183/OI_SiII/CD/Halo1/histo_tot_O_%i.png'%k)
    plt.close()

In [10]:
#SiII column density :
for k in range(ndirections):

    CD = f.read_reals()
    #CD begins with npart CD for the first bin of velocity,  then npart CD for the second bin,  etc

    CD[np.where(CD>0)] = np.log10(CD[np.where(CD>0)])

    #Question : 
    #for i in range(nrays):
    #    weights[i] = weights[i]/np.where(CD2[:,i]!=0)[0].size
    
    v_all = np.repeat(v,nrays)
    weights_all = np.tile(weights,nBins+2)

    xedges = np.linspace(vmin-dv, vmax + dv,nBins+3)
    yedges = np.arange(1,21.1,0.8)

    histo, xedges, yedges = np.histogram2d(v_all, CD, bins=(xedges,yedges), weights=weights_all, normed=False)

    for i in range(histo[:,0].size):
        histo[i,:] = histo[i,:]/sum(histo[i,:])

    histo = np.ma.masked_where(histo < 0.01, histo)
    cmap = plt.cm.gist_ncar_r
    #cmap = plt.cm.PuBu_r
    cmap.set_bad(color='white')
    #cmap.set_bad(color='black')

    fig, ax = plt.subplots(figsize=(10, 6))
    plt.imshow(histo.T, origin='lower', extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]], aspect='auto', cmap=cmap)#,norm=LogNorm())
    plt.axhline(14.5,color='purple',linestyle='--', label=r'$\tau_{1302} = 10$')
    plt.axhline(13.5,color='deeppink',linestyle='--', label=r'$\tau_{1302} = 1$')
    plt.axvline(0,color='limegreen',linestyle='--')
    plt.legend(loc='upper left', fontsize=14)
    cbar = plt.colorbar()
    plt.clim(0,1)
    plt.xlabel('v [km/s]', fontsize=14)
    plt.ylabel(r'$\mathrm{log}( \: N_{SiII} \, [\mathrm{cm}^{-2}] \: )$', fontsize=14)
    #plt.title('2D histogram, direction %s'%directions[k], fontsize = 16)
    plt.title('2D histogram, direction %05.4f %05.4f %05.4f'%(directions[k,0],directions[k,1],directions[k,2]), fontsize = 16)
    cbar.set_label('Fraction of luminosity', fontsize=14)
    plt.xticks(np.arange(xedges[0]+dv, xedges[-1]-dv+0.01, 150))
    #plt.show()
    plt.savefig('/cral/mauerhofer/RascasFiles/sphinx/05_F1000/02_IC20_BP/00183/OI_SiII/CD/Halo1/histo_tot_Si_%i.png'%k)
    plt.close()

In [11]:
#dust column density :
for k in range(ndirections):

    CD = f.read_reals()
    #CD begins with npart CD for the first bin of velocity,  then npart CD for the second bin,  etc

    CD[np.where(CD>0)] = np.log10(CD[np.where(CD>0)])

    #Question : 
    #for i in range(nrays):
    #    weights[i] = weights[i]/np.where(CD2[:,i]!=0)[0].size
    
    v_all = np.repeat(v,nrays)
    weights_all = np.tile(weights,nBins+2)

    xedges = np.linspace(vmin-dv, vmax + dv,nBins+3)
    yedges = np.arange(1,21.1,0.8)

    histo, xedges, yedges = np.histogram2d(v_all, CD, bins=(xedges,yedges), weights=weights_all, normed=False)

    for i in range(histo[:,0].size):
        histo[i,:] = histo[i,:]/sum(histo[i,:])

    histo = np.ma.masked_where(histo < 0.01, histo)
    cmap = plt.cm.gist_ncar_r
    #cmap = plt.cm.PuBu_r
    cmap.set_bad(color='white')
    #cmap.set_bad(color='black')

    fig, ax = plt.subplots(figsize=(10, 6))
    plt.imshow(histo.T, origin='lower', extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]], aspect='auto', cmap=cmap)#,norm=LogNorm())
    plt.axhline(14.5,color='purple',linestyle='--', label=r'$\tau_{1302} = 10$')
    plt.axhline(13.5,color='deeppink',linestyle='--', label=r'$\tau_{1302} = 1$')
    plt.axvline(0,color='limegreen',linestyle='--')
    plt.legend(loc='upper left', fontsize=14)
    cbar = plt.colorbar()
    plt.clim(0,1)
    plt.xlabel('v [km/s]', fontsize=14)
    plt.ylabel(r'$\mathrm{log}( \: N_{dust} \, [\mathrm{cm}^{-2}] \: )$', fontsize=14)
    #plt.title('2D histogram, direction %s'%directions[k], fontsize = 16)
    plt.title('2D histogram, direction %05.4f %05.4f %05.4f'%(directions[k,0],directions[k,1],directions[k,2]), fontsize = 16)
    cbar.set_label('Fraction of luminosity', fontsize=14)
    plt.xticks(np.arange(xedges[0]+dv, xedges[-1]-dv+0.01, 150))
    #plt.show()
    plt.savefig('/cral/mauerhofer/RascasFiles/sphinx/05_F1000/02_IC20_BP/00183/OI_SiII/CD/Halo1/histo_tot_dust_%i.png'%k)
    plt.close()

In [34]:
print(plt.style.available)

['Solarize_Light2', '_classic_test', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-bright', 'seaborn-colorblind', 'seaborn-dark-palette', 'seaborn-dark', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'seaborn', 'tableau-colorblind10']
